In [50]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import time

In [92]:
def get_team_df(dict_correspondances, countries):
    player_name, player_country, player_position, player_age, player_description, player_link = [], [], [], [], [], []
    browser = webdriver.Firefox()
    browser.get("https://www.fifa.com/worldcup/teams/")
    time.sleep(1)
    
    for country in countries:
        element = browser.find_elements_by_xpath("//div[@class='fi-team-card__flag']")[dict_correspondances[country]]
        browser.execute_script("arguments[0].click();", element)
        time.sleep(1)

        for p in range(23):
            player = browser.find_elements_by_xpath("//div[@class='fi-p']/a[@class='fi-p--link']")[p]
            new_url = player.get_attribute('href')
            browser.get(new_url)
            time.sleep(1)
            player_name.append(browser.find_element_by_xpath("//div[@class='fi-p__name']").text)
            print(player_name[-1])
            player_country.append(country)
            player_position.append(browser.find_element_by_xpath("//div[@class='fi-p__role']").text)
            player_age.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-number__number']").text)
            try:
                player_description.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-text']").text)
            except NoSuchElementException:
                player_description.append("")
            my_img = browser.find_elements_by_tag_name('image')
            for image in my_img:
                link = image.get_attribute('xlink:href')
                player_link.append(link)
            browser.back()
            time.sleep(1)

        browser.get("https://www.fifa.com/worldcup/teams/")
        time.sleep(2)
    
    df = pd.DataFrame({"Name":player_name, "Country":player_country, "Position":player_position, "Age":player_age, 
                   "Description":player_description, "Link":player_link})
    df["Firstname"] = df["Name"].apply(lambda x: x.split()[0].capitalize())
    df["Lastname"] = df["Name"].apply(lambda x: "".join(x.split()[1:]).capitalize())
    return df

dict_correspondances = {"Australie": 0, "Belgique": 2, "Bresil": 3,
                        "Colombie": 4, "Costa": 5, "Croatie": 6,
                        "Danemark": 7, "Egypte": 8, "Angleterre": 9,
                        "France": 10, "Allemagne": 11, "Iran": 13,
                        "Maroc": 17, "Perou": 20, "Pologne": 21,
                        "Portugal": 22, "Russie": 23, "Arabie": 24,
                        "Serbie": 26, "Espagne": 27, "Suede": 28,
                        "Suisse": 29, "Tunisie": 30}

df_rusara = get_team_df(dict_correspondances, ["Russie", "Arabie"])

IGOR AKINFEEV
ANDREI LUNEV
VLADIMIR GABULOV
MARIO FERNANDES
ILYA KUTEPOV
SERGEI IGNASHEVICH
ANDREY SEMENOV
FEDOR KUDRIASHOV
VLADIMIR GRANAT
IGOR SMOLNIKOV
DENIS CHERYSHEV
DALER KUZIAEV
IURY GAZINSKY
ALAN DZAGOEV
ROMAN ZOBNIN
ALEKSEY MIRANCHUK
ANTON MIRANCHUK
ALEKSANDR GOLOVIN
YURI ZHIRKOV
ALEKSANDR SAMEDOV
ALEKSANDR EROKHIN
FEDOR SMOLOV
ARTEM DZYUBA
ABDULLAH ALMUAIOUF
YASSER AL MOSAILEM
MOHAMMED ALOWAIS
MANSOUR ALHARBI
OSAMA HAWSAWI
ALI ALBULAYHI
OMAR HAWSAWI
MOHAMMED ALBURAYK
YASIR AL SHAHRANI
MOTAZ HAWSAWI
SALMAN AL FARAJ
YAHIA AL SHEHRI
HATAN BAHBIR
ABDULMALEK AL KHAIBRI
MOHAMED KANNO
ABDULLAH OTAYF
ABDULLAH ALKHAIBARI
HUSSAIN AL MOQAHWI
TAISEER AL JASSAM
SALEM AL DAWSARI
MOHAMMED ALSAHLAWI
FAHAD AL MUWALLAD
MUHANNAD ABU RADIYAH


In [93]:
df_rusara

,Age,Country,Description,Link,Name,Position,Firstname,Lastname
0,32,Russie,"Born in Vidnoe near the capital, Igor Akinfeev...",https://api.fifa.com/api/v1/picture/players/20...,IGOR AKINFEEV,Goalkeeper,Igor,Akinfeev
1,26,Russie,Andrey Lunyov should have made his bow on the ...,https://api.fifa.com/api/v1/picture/players/20...,ANDREI LUNEV,Goalkeeper,Andrei,Lunev
2,34,Russie,"Born in Mozdok in North Ossetia, Vladimir Gabu...",https://api.fifa.com/api/v1/picture/players/20...,VLADIMIR GABULOV,Goalkeeper,Vladimir,Gabulov
3,27,Russie,"After five years in the country, Mario Fernand...",https://api.fifa.com/api/v1/picture/players/20...,MARIO FERNANDES,Defender,Mario,Fernandes
4,24,Russie,Central defender Ilya Kutepov started out at h...,https://api.fifa.com/api/v1/picture/players/20...,ILYA KUTEPOV,Defender,Ilya,Kutepov
5,38,Russie,Only CSKA Moscow team-mate Igor Akinfeev could...,https://api.fifa.com/api/v1/picture/players/20...,SERGEI IGNASHEVICH,Defender,Sergei,Ignashevich
6,29,Russie,Stanislav Cherchesov picked Andrey Semenov for...,https://api.fifa.com/api/v1/picture/players/20...,ANDREY SEMENOV,Defender,Andrey,Semenov
7,31,Russie,Stanislav Cherchesov handed Kudriashov a start...,https://api.fifa.com/api/v1/picture/players/20...,FEDOR KUDRIASHOV,Defender,Fedor,Kudriashov
8,31,Russie,Vladimir Granat started playing football aged ...,https://api.fifa.com/api/v1/picture/players/20...,VLADIMIR GRANAT,Defender,Vladimir,Granat
9,29,Russie,Igor Smolnikov first became a regular presence...,https://api.fifa.com/api/v1/picture/players/20...,IGOR SMOLNIKOV,Defender,Igor,Smolnikov


In [97]:
def image_downloader(name, team, link):
    with open(team.lower() + '/' + name.lower() + '.jpg', 'wb') as handle:
            response = requests.get(link, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)

for n, t, l in zip(df["Lastname"].tolist(), df["Country"].tolist(), df["Link"].tolist()):
    image_downloader(n, t, l)